In [1]:
%load_ext sparksql_magic
from pyspark.sql import SparkSession

# Start Spark session
spark = (
    SparkSession.builder.appName("FilterGHJsonPYTHON")
    .config("spark.sql.catalogImplementation", "hive")
    .enableHiveSupport()
    .getOrCreate()
)

your 131072x1 screen size is bogus. expect trouble
25/03/09 21:43:23 WARN Utils: Your hostname, NathanBlade15 resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/03/09 21:43:23 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/09 21:43:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
from pyspark.sql.functions import col
df = spark.read.json("../data/*.json.gz")
# Filter for IssuesEvent
df = df.filter(df["type"] == "IssuesEvent")
df = df.select(
    col("actor"),
    col("created_at"),
    col("id").alias("event_id"),
    col("org"),
    col("payload.action"),
    col("payload.issue")
)

25/03/09 21:43:52 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [3]:
df.createOrReplaceTempView("df")
df.show()

+--------------------+--------------------+-----------+--------------------+--------+--------------------+
|               actor|          created_at|   event_id|                 org|  action|               issue|
+--------------------+--------------------+-----------+--------------------+--------+--------------------+
|{https://avatars....|2025-01-01T01:00:02Z|45186096107|                NULL|  opened|{NULL, NULL, [], ...|
|{https://avatars....|2025-01-01T01:00:02Z|45186096173|                NULL|  opened|{NULL, NULL, [], ...|
|{https://avatars....|2025-01-01T01:00:04Z|45186096455|                NULL|  opened|{NULL, NULL, [], ...|
|{https://avatars....|2025-01-01T01:00:04Z|45186096537|                NULL|  closed|{NULL, NULL, [], ...|
|{https://avatars....|2025-01-01T01:00:04Z|45186096631|{https://avatars....|  opened|{NULL, NULL, [], ...|
|{https://avatars....|2025-01-01T01:00:08Z|45186097554|                NULL|  opened|{NULL, NULL, [], ...|
|{https://avatars....|2025-01-01T01:0

In [33]:
spark.sql(""" select issue.* from df """).show()


+------------------+--------------------+--------------------+------------------+-----------------------------+--------------------+--------+--------------------+--------------------+-----+--------------------+--------------------+----------+--------------------+--------------------+------+---------+------------------+------+------------------------+------------+--------------------+--------------------+------+------------+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|active_lock_reason|            assignee|           assignees|author_association|                         body|           closed_at|comments|        comments_url|          created_at|draft|          events_url|            html_url|        id|              labels|          labels_url|locked|milestone|           node_id|number|performed_via_github_app|pull_request|           reactions|      repository_url| state|state_reason|sub_issues_summary|  

## Build up the issues scd type II table

In [ ]:
%%sparksql
Create table if not exists scd_issues(
    user_id BIGINT,
    event_id BIGINT,
    org_id as BIGINT,
    action VARCHAR(8),
    created_at TIMESTAMP
)

In [20]:
df = spark.sql("""
    SELECT 
        issue.id AS issue_id,
        issue.author_association AS author_role,
        issue.title AS issue_title,
        issue.body AS issue_description,
        issue.closed_at AS closed_timestamp,
        issue.created_at AS created_timestamp,
        issue.state AS issue_state,
        issue.comments AS comment_count,
        issue.state_reason,
        issue.updated_at
    FROM df
""")

# issue.state_reason is always NOT NULL when issue.state is 'closed' 
# otherwise it is Always NULL

In [22]:
df.createOrReplaceTempView("df")
df.show()

+----------+------------+--------------------+-----------------------------+--------------------+--------------------+-----------+-------------+------------+--------------------+
|  issue_id| author_role|         issue_title|            issue_description|    closed_timestamp|   created_timestamp|issue_state|comment_count|state_reason|          updated_at|
+----------+------------+--------------------+-----------------------------+--------------------+--------------------+-----------+-------------+------------+--------------------+
|2764758647|       OWNER|Check if time is ...|         Please check if t...|                NULL|2025-01-01T01:00:00Z|       open|            0|        NULL|2025-01-01T01:00:00Z|
|2764758656|        NONE|it's me again , c...|         Since too many ne...|                NULL|2025-01-01T01:00:01Z|       open|            0|        NULL|2025-01-01T01:00:01Z|
|2764758662|       OWNER|Fixing HR 0x80072...|         \nThe HR 0x80072E...|                NULL|2025-01-

In [ ]:
spark.sql("""
    WITH prev_state as (
          SELECT 
            *, 
            lag(issue_state) over (partition by issue_id order by updated_at) as prev_issue_state,
            lag(state_reason) over (partition by issue_id order by updated_at) as prev_state_reason,
            lag(closed_timestamp) over (partition by issue_id order by updated_at) as prev_closed_timestamp,
          FROM df
          ),
          change_indicator as (
        select 
          *, 
          CASE WHEN (issue_state != prev_issue_state OR state_reason != prev_state_reason OR closed_timestamp != prev_closed_timestamp) THEN 1 ELSE 0 END as changed
        from prev_state
          ),
        streaks as (
          select 
            *, 
            sum(changed) over (partition by issue_id order by updated_at) as streak
          from change_indicator
          ) 
          select issue_id,author_role,issue_title,issue_description,created_timestamp,closed_timestamp from streaks
""").show()

+---------+------------+----------------------------------------+-----------------------------------+--------------------+--------------------+-----------+-------------+------------+--------------------+----------------+-----------------+-------+------+
| issue_id| author_role|                             issue_title|                  issue_description|    closed_timestamp|   created_timestamp|issue_state|comment_count|state_reason|          updated_at|prev_issue_state|prev_state_reason|changed|streak|
+---------+------------+----------------------------------------+-----------------------------------+--------------------+--------------------+-----------+-------------+------------+--------------------+----------------+-----------------+-------+------+
|234049579|        NONE|                    Google Assistant ...|               Hey, great timer!...|2025-01-01T18:01:39Z|2017-06-06T23:01:43Z|     closed|            0|   completed|2025-01-01T18:01:39Z|            NULL|             NULL|

In [19]:
spark.sql("""
    SELECT 
        issue.id AS issue_id,
        action,
        issue.author_association AS author_role,
        issue.title AS issue_title,
        issue.body AS issue_description,
        CAST(issue.closed_at AS TIMESTAMP) AS closed_timestamp,
        CAST(issue.created_at AS TIMESTAMP) AS created_timestamp,
        issue.state AS issue_state,
        issue.comments AS comment_count,
        issue.state_reason,
        
        -- Convert created_at and closed_at to UNIX timestamps properly
        ROUND(
            (CASE 
                WHEN issue.closed_at IS NOT NULL 
                THEN unix_timestamp(CAST(issue.closed_at AS TIMESTAMP)) 
                     - unix_timestamp(CAST(issue.created_at AS TIMESTAMP))
                ELSE unix_timestamp(current_timestamp()) 
                     - unix_timestamp(CAST(issue.created_at AS TIMESTAMP))
            END) / 3600, 2
        ) AS time_open_hours

    FROM df
          WHERE issue.state = 'closed'
          AND issue.closed_at IS NOT NULL
          AND issue.created_at IS NOT NULL
          AND issue.closed_at > issue.created_at
""").show()


+----------+------+------------+--------------------------+-----------------------------+-------------------+-------------------+-----------+-------------+------------+---------------+
|  issue_id|action| author_role|               issue_title|            issue_description|   closed_timestamp|  created_timestamp|issue_state|comment_count|state_reason|time_open_hours|
+----------+------+------------+--------------------------+-----------------------------+-------------------+-------------------+-----------+-------------+------------+---------------+
|2764728177|closed|        NONE|      Pumps turning off...|         When I use the ph...|2024-12-31 20:00:03|2024-12-31 18:42:35|     closed|            3|   completed|           1.29|
|2747178133|closed|       OWNER|              ヘッダー作成|## 実装チェックリスト\n## ...|2024-12-31 20:00:13|2024-12-18 03:46:44|     closed|            0|   completed|         328.22|
|2761657545|closed|        NONE|      Mcprep crush on b...|         ### Did the issue...|2

In [20]:
spark.sql("""
    SELECT 
        issue.id AS issue_id,
        action,
        issue.author_association AS author_role,
        issue.title AS issue_title,
        issue.body AS issue_description,
        CAST(issue.closed_at AS TIMESTAMP) AS closed_timestamp,
        CAST(issue.created_at AS TIMESTAMP) AS created_timestamp,
        issue.state AS issue_state,
        issue.comments AS comment_count,
        issue.state_reason,
        
        -- Convert created_at and closed_at to UNIX timestamps properly
        ROUND(
            (CASE 
                WHEN issue.closed_at IS NOT NULL 
                THEN unix_timestamp(CAST(issue.closed_at AS TIMESTAMP)) 
                     - unix_timestamp(CAST(issue.created_at AS TIMESTAMP))
                ELSE unix_timestamp(current_timestamp()) 
                     - unix_timestamp(CAST(issue.created_at AS TIMESTAMP))
            END) / 3600, 2
        ) AS time_open_hours

    FROM df
          WHERE issue.state = 'open'
          AND issue.created_at IS NOT NULL
""").show()


+----------+--------+------------+--------------------+-----------------------------+----------------+-------------------+-----------+-------------+------------+---------------+
|  issue_id|  action| author_role|         issue_title|            issue_description|closed_timestamp|  created_timestamp|issue_state|comment_count|state_reason|time_open_hours|
+----------+--------+------------+--------------------+-----------------------------+----------------+-------------------+-----------+-------------+------------+---------------+
|2764758647|  opened|       OWNER|Check if time is ...|         Please check if t...|            NULL|2024-12-31 20:00:00|       open|            0|        NULL|        1586.02|
|2764758656|  opened|        NONE|it's me again , c...|         Since too many ne...|            NULL|2024-12-31 20:00:01|       open|            0|        NULL|        1586.02|
|2764758662|  opened|       OWNER|Fixing HR 0x80072...|         \nThe HR 0x80072E...|            NULL|2024-12-

In [22]:
spark.sql("""
    SELECT *
    FROM (
        SELECT 
            issue.id AS issue_id,
            action,
            issue.author_association AS author_role,
            issue.title AS issue_title,
            issue.body AS issue_description,
            CAST(issue.closed_at AS TIMESTAMP) AS closed_timestamp,
            CAST(issue.created_at AS TIMESTAMP) AS created_timestamp,
            issue.state AS issue_state,
            issue.state_reason,
            
            -- Count occurrences of the same record
            COUNT(*) OVER (
                PARTITION BY issue.id, action, issue.author_association, 
                issue.title, issue.body, issue.closed_at, 
                issue.created_at, issue.state, issue.state_reason
            ) AS duplicate_count
            
        FROM df
    )
    WHERE duplicate_count > 1
    ORDER BY issue_id, created_timestamp DESC
""").show()


+----------+------+------------+--------------------+--------------------+-------------------+-------------------+-----------+------------+---------------+
|  issue_id|action| author_role|         issue_title|   issue_description|   closed_timestamp|  created_timestamp|issue_state|state_reason|duplicate_count|
+----------+------+------------+--------------------+--------------------+-------------------+-------------------+-----------+------------+---------------+
| 175296384|closed| CONTRIBUTOR|JavaScript: destr...|When playing with...|2025-01-01 00:06:51|2016-09-06 12:49:17|     closed|   completed|              2|
| 175296384|closed| CONTRIBUTOR|JavaScript: destr...|When playing with...|2025-01-01 00:06:51|2016-09-06 12:49:17|     closed|   completed|              2|
| 360646678|closed|COLLABORATOR|setElementHealth ...|**Describe the bu...|2025-01-01 14:09:20|2018-09-16 10:53:51|     closed|   completed|              2|
| 360646678|closed|COLLABORATOR|setElementHealth ...|**Describe 

In [28]:
spark.sql("""
    WITH base_data AS (
        -- Step 1: Select relevant columns from df (raw data)
        SELECT 
            issue.id AS issue_id,
            action,
            issue.author_association AS author_role,
            issue.title AS issue_title,
            issue.body AS issue_description,
            CAST(issue.closed_at AS TIMESTAMP) AS closed_timestamp,
            CAST(issue.created_at AS TIMESTAMP) AS created_timestamp,
            issue.state AS issue_state,
            issue.state_reason
        FROM df
    ),

    deduped_data AS (
        -- Step 2: Remove exact duplicate rows (same issue, same event details)
        SELECT DISTINCT * FROM base_data
    ),

    lagged_data AS (
        -- Step 3: Use LAG() to get the previous state for each issue_id
        SELECT *,
            LAG(issue_state) OVER (PARTITION BY issue_id ORDER BY created_timestamp) AS prev_state
        FROM deduped_data
    ),

    filtered_data AS (
        -- Step 4: Filter redundant logs (ignore single-event issues)
        SELECT *
        FROM lagged_data
        WHERE prev_state IS NOT NULL  -- Exclude single-event issues
        AND issue_state = prev_state  -- Keep only redundant logs
    )

    -- Step 5: Return the cleaned redundant logs
    SELECT * FROM filtered_data
    ORDER BY issue_id, created_timestamp;
""").show()


+----------+--------+------------+----------------------------+--------------------+-------------------+-------------------+-----------+------------+----------+
|  issue_id|  action| author_role|                 issue_title|   issue_description|   closed_timestamp|  created_timestamp|issue_state|state_reason|prev_state|
+----------+--------+------------+----------------------------+--------------------+-------------------+-------------------+-----------+------------+----------+
| 424486938|  closed|        NONE|        -readingOrder opt...|Hi,\r\n\r\nin the...|2025-01-01 02:55:22|2019-03-23 06:37:12|     closed| not_planned|    closed|
| 465859330|  closed|      MEMBER|              Support Codacy|https://support.c...|2025-01-01 13:47:31|2019-07-09 11:50:51|     closed| not_planned|    closed|
|1462633089|  closed|COLLABORATOR|        ADD bert-base-jap...|https://hakasenot...|2025-01-01 08:08:27|2022-11-23 20:39:23|     closed|   completed|    closed|
|1524487483|  closed|COLLABORATOR|

In [35]:
spark.sql("""
    WITH base_data AS (
        -- Step 1: Extract relevant columns
        SELECT 
            issue.id AS issue_id,
            action,
            issue.author_association AS author_role,
            issue.title AS issue_title,
            issue.body AS issue_description,
            CAST(issue.closed_at AS TIMESTAMP) AS closed_timestamp,
            CAST(issue.created_at AS TIMESTAMP) AS created_timestamp,
            CAST(issue.updated_at AS TIMESTAMP) AS updated_timestamp,  -- Use updated_at to track event order
            issue.state AS issue_state,
            issue.state_reason
        FROM df
    ),

    deduped_data AS (
        -- Step 2: Remove exact duplicate rows
        SELECT DISTINCT * FROM base_data
    ),

    lagged_data AS (
        -- Step 3: Use LAG() with updated_at to track previous state
        SELECT *,
            LAG(issue_state) OVER (PARTITION BY issue_id ORDER BY updated_timestamp) AS prev_state,
            
            -- Flag whether the issue_state is the same as the previous one
            CASE 
                WHEN issue_state = LAG(issue_state) OVER (PARTITION BY issue_id ORDER BY updated_timestamp) 
                THEN 1 ELSE 0 
            END AS is_redundant_log

        FROM deduped_data
    )

    -- Step 4: Show full history, including flagged redundant logs
    SELECT * 
    FROM lagged_data
    ORDER BY issue_id, updated_timestamp;
""").show()


+---------+------+------------+--------------------+--------------------+-------------------+-------------------+-------------------+-----------+------------+----------+----------------+
| issue_id|action| author_role|         issue_title|   issue_description|   closed_timestamp|  created_timestamp|  updated_timestamp|issue_state|state_reason|prev_state|is_redundant_log|
+---------+------+------------+--------------------+--------------------+-------------------+-------------------+-------------------+-----------+------------+----------+----------------+
| 46169398|closed| CONTRIBUTOR|           noise.png|The image noise.p...|2025-01-01 10:43:15|2014-10-18 05:53:48|2025-01-01 10:43:16|     closed|   completed|      NULL|               0|
| 46826622|closed| CONTRIBUTOR|How `box` should ...|We need to come u...|2025-01-01 18:16:01|2014-10-25 19:22:26|2025-01-01 18:16:02|     closed|   completed|      NULL|               0|
| 57766158|closed| CONTRIBUTOR|Design the Spyyn SPA|We need some 

In [37]:
spark.sql("""
    WITH base_data AS (
        -- Step 1: Extract relevant columns
        SELECT 
            issue.id AS issue_id,
            action,
            issue.author_association AS author_role,
            issue.title AS issue_title,
            issue.body AS issue_description,
            CAST(issue.closed_at AS TIMESTAMP) AS closed_timestamp,
            CAST(issue.created_at AS TIMESTAMP) AS created_timestamp,
            CAST(issue.updated_at AS TIMESTAMP) AS updated_timestamp,  -- Use updated_at for event tracking
            issue.state AS issue_state
            -- issue.state_reason
        FROM df
    ),

    deduped_data AS (
        -- Step 2: Remove exact duplicate rows
        SELECT DISTINCT * FROM base_data
    ),

    lagged_data AS (
        -- Step 3: Use LAG() to detect redundant logs
        SELECT *,
            LAG(issue_state) OVER (PARTITION BY issue_id ORDER BY updated_timestamp) AS prev_state,
            
            -- Flag redundant logs where issue_state is the same as the last one
            CASE 
                WHEN issue_state = LAG(issue_state) OVER (PARTITION BY issue_id ORDER BY updated_timestamp) 
                THEN 1 ELSE 0 
            END AS is_redundant_log

        FROM deduped_data
    )

    -- Step 4: Show full event history while tracking total redundant logs per issue
    SELECT *,
        SUM(is_redundant_log) OVER (PARTITION BY issue_id) AS total_redundant_events  -- Running total per issue
    FROM lagged_data
    ORDER BY total_redundant_events DESC, issue_id, updated_timestamp;
""").show()


+----------+------+------------+--------------------+--------------------+-------------------+-------------------+-------------------+-----------+----------+----------------+----------------------+
|  issue_id|action| author_role|         issue_title|   issue_description|   closed_timestamp|  created_timestamp|  updated_timestamp|issue_state|prev_state|is_redundant_log|total_redundant_events|
+----------+------+------------+--------------------+--------------------+-------------------+-------------------+-------------------+-----------+----------+----------------+----------------------+
|2763306076|closed|        NONE|                   .|                NULL|2024-12-30 09:45:14|2024-12-30 09:43:51|2025-01-01 08:58:29|     closed|      NULL|               0|                    10|
|2763306076|closed|        NONE|                   .|                NULL|2024-12-30 09:45:14|2024-12-30 09:43:51|2025-01-01 08:58:32|     closed|    closed|               1|                    10|
|276330607